In [15]:
#dynamo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import scanpy as scx
import scvelo as scv 
import sys
import os
import dynamo as dyn
import re

In [44]:
adata = scv.read('scvelo_oscc.loom')
positions = pd.read_csv("positions_scvelo.csv")
positions = positions.tail(positions.shape[0] -1)
positions.index = positions.index.rename("CellID")
positions["CellID"] = positions["Barcodes"]
positions.drop("Barcodes", inplace=True, axis=1)
positions.index = positions["CellID"]
positions['UMAP_1'] = positions['UMAP_1'].astype('float32')
positions['UMAP_2'] = positions['UMAP_2'].astype('float32')

adata.obs = adata.obs.join(positions, how="left")

adata.obsm['X_umap'] = adata.obs[
            ['UMAP_1', 'UMAP_2']
        ].to_numpy()

adata.obs.drop(
            columns=['CellID', 'UMAP_1', 'UMAP_2'],
            inplace=True,
        )

meta = pd.read_csv("scvelo_metadata_headers.csv")
meta = meta.tail(meta.shape[0] -1)
meta["CellID"] = meta["Barcode"]
meta.drop("Barcode", inplace=True, axis=1)
meta.index = meta["CellID"]
adata.obs = adata.obs.join(meta, how="left")
adata = adata[adata.obs["orig.ident"] == "ST_OSCC"]

dyn.pp.recipe_monocle(adata,genes_to_append = ['CTLA4'])
dyn.tl.dynamics(adata, cores=9)

dyn.tl.reduceDimension(adata)

dyn.tl.cell_velocities(adata, basis='pca')
dyn.tl.cell_velocities(adata, basis='umap')

dyn.vf.VectorField(adata,
               basis='pca',
               M=100)

dyn.vf.VectorField(adata,
               basis='umap',
               M=100)

|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...
|-----> <insert> pp to uns in AnnData Object.
|-----------> <insert> has_splicing to uns['pp'] in AnnData Object.
|-----------> <insert> has_labling to uns['pp'] in AnnData Object.
|-----------> <insert> splicing_labeling to uns['pp'] in AnnData Object.
|-----------> <insert> has_protein to uns['pp'] in AnnData Object.
|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----------> <insert> tkey to uns['

In [ ]:
#baseline vf plot
dyn.pl.streamline_plot(adata, basis = "umap",
                          color=["cluster_annotations"],
                          color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
                          show_legend = False,
                          calpha = 1,
                          pointsize = 0.1,
                          save_show_or_return = "save",
                           save_kwargs = {"path": f"plots/vector_dynamo", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
                           "transparent": False, "close": True, "verbose": True},
                          figsize=(5,5),
                          dpi = 300)

In [45]:
#baseline State Graph plot
dyn.pd.state_graph(adata, group='cluster_annotations', basis='umap')
dyn.pl.state_graph(adata,
                   color=['cluster_annotations'],
                   color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
                   group='cluster_annotations',
                   basis='umap',
                   keep_only_one_direction = False,
                   save_show_or_return = "save",figsize=(5,5),
                   save_kwargs = {"path": f"plots/baseline_state_graph", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
                   "transparent": False, "close": True, "verbose": True})

|-----> Estimating the transition probability between cell types...
|-----> Applying vector field
|-----> [KDTree computation] in progress: 100.0000%in progress: 0.0000%
|-----> [KDTree computation] finished [0.0051s]
|-----> [iterate groups] in progress: 33.3333%

uniformly sampling points along a trajectory: 100%|██████| 100/100 [00:00<00:00, 261.80it/s]


|-----> [iterate groups] in progress: 66.6667%

uniformly sampling points along a trajectory: 100%|██████| 100/100 [00:00<00:00, 247.90it/s]


|-----> [iterate groups] in progress: 100.0000%

uniformly sampling points along a trajectory: 100%|██████| 100/100 [00:00<00:00, 248.54it/s]


|-----> [iterate groups] in progress: 100.0000%
|-----> [iterate groups] finished [15.1033s]
|-----> [State graph estimation] in progress: 100.0000%
|-----> [State graph estimation] finished [0.0013s]
|-----------> plotting with basis key=X_umap
|-----------> skip filtering cluster_annotations by stack threshold when stacking color because it is not a numeric type
Saving figure to plots/scatter_baseline_state_graph.png...
Done


In [ ]:
#Immunotherapy plots
dyn.pd.perturbation(adata, ["CD274"], -1000, emb_basis="umap")
dyn.vf.VectorField(adata,
           basis='umap_perturbation',
           M=100)

dyn.pl.streamline_plot(adata, basis = "umap_perturbation",
                          color=["cluster_annotations"],
                          color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
                          show_legend = False,
                          calpha = 1,
                          pointsize = 0.1,
                          save_show_or_return = "save",
                           save_kwargs = {"path": f"plots/immuno/CD274", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
                           "transparent": False, "close": True, "verbose": True},
                          figsize=(5,5),
                          dpi = 300)


dyn.pd.state_graph(adata, group='cluster_annotations', basis='umap_perturbation',
      transition_mat_key = 'perturbation_transition_matrix')
dyn.pl.state_graph(adata,
       color=['cluster_annotations'],
       color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
       group='cluster_annotations',
       basis='umap_perturbation',
       save_show_or_return='save',
       save_kwargs = {"path": f"plots/immuno/CD274_sg", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
       "transparent": False, "close": True, "verbose": True},
        keep_only_one_direction = False,
          figsize=(5,5))

In [ ]:
#Immunotherapy plots
dyn.pd.perturbation(adata, ["CTLA4"], -1000, emb_basis="umap")
dyn.vf.VectorField(adata,
           basis='umap_perturbation',
           M=100)

dyn.pl.streamline_plot(adata, basis = "umap_perturbation",
                          color=["cluster_annotations"],
                          color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
                          show_legend = False,
                          calpha = 1,
                          pointsize = 0.1,
                          save_show_or_return = "save",
                           save_kwargs = {"path": f"plots/immuno/CTLA4", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
                           "transparent": False, "close": True, "verbose": True},
                          figsize=(5,5),
                          dpi = 300)

dyn.pd.state_graph(adata, group='cluster_annotations', basis='umap_perturbation', 
      transition_mat_key = 'perturbation_transition_matrix')
dyn.pl.state_graph(adata,
       color=['cluster_annotations'],
       color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
       group='cluster_annotations',
       basis='umap_perturbation',
       save_show_or_return='save',
       save_kwargs = {"path": f"plots/immuno/CTLA4_sg", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
       "transparent": False, "close": True, "verbose": True},
        keep_only_one_direction = False,
          figsize=(5,5))

In [ ]:
#base state (t,e,c)
adata.uns["cluster_annotations_graph"]

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import dynamo as dyn
import re

|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


In [ ]:
#although much of the below code was labeled as ic50, it was conducted with AAC values

In [1]:
ic50_dgidb = pd.read_csv("targeted_survival/dgidb_AUC_values.csv", index_col= 0)

NameError: name 'pd' is not defined

In [8]:
ic50_dgidb = pd.read_csv("targeted_survival/dgidb_AUC_values.csv", index_col= 0)
genes_down = set(re.sub(' +', ' ', ic50_dgidb['genes_down'].to_string(index=False).strip().replace('\n', ' ').replace('NaN', '')).split(" "))
genes_down = list(genes_down)
genes_up = set(re.sub(' +', ' ', ic50_dgidb['genes_up'].to_string(index=False).strip().replace('\n', ' ').replace('NaN', '')).split(" "))
genes_up = list(genes_up)
all_genes = genes_down + genes_up
all_genes.remove('')
ic50_dgidb = ic50_dgidb.rename(index=ic50_dgidb.iloc[0:len(ic50_dgidb)]['drugs'])
ic50_dgidb["pertrub_mtx"] = [[0,0,0,0,0,0,0,0,0]] * len(ic50_dgidb)
#remove nan rows
ic50_dgidb = ic50_dgidb.dropna(subset=['genes_up', 'genes_down'], how='all')

def process_data(ic50_dgidb, all_genes):
    # Read the data outside the loop
    adata_template = scv.read('scvelo_oscc.loom')
    positions = pd.read_csv("positions_scvelo.csv")
    meta = pd.read_csv("scvelo_metadata_headers.csv")

    # Preprocess the data outside the loop
    adata_template = preprocess_data(adata_template, positions, meta)
    ic50_dgidb_scored = ic50_dgidb.copy()
    for i in ic50_dgidb.index:
        adata_pt = adata_template.copy()
        genes_up, genes_down, nz_valid_genes = get_genes(adata_pt, ic50_dgidb, i, all_genes)
        ic50_dgidb_scored = run_perturbation_analysis(adata_pt, genes_up, genes_down, nz_valid_genes, i, ic50_dgidb, ic50_dgidb_scored)
    return ic50_dgidb_scored

def preprocess_data(adata_template, positions, meta):
    # Preprocess positions DataFrame
    positions = positions.tail(positions.shape[0] -1)
    positions.index = positions.index.rename("CellID")
    positions["CellID"] = positions["Barcodes"]
    positions.drop("Barcodes", inplace=True, axis=1)
    positions.index = positions["CellID"]
    positions['UMAP_1'] = positions['UMAP_1'].astype('float32')
    positions['UMAP_2'] = positions['UMAP_2'].astype('float32')

    adata_template.obs = adata_template.obs.join(positions, how="left")
    adata_template.obsm['X_umap'] = adata_template.obs[['UMAP_1', 'UMAP_2']].to_numpy()
    adata_template.obs.drop(columns=['CellID', 'UMAP_1', 'UMAP_2'], inplace=True)

    # Preprocess meta DataFrame
    meta = meta.tail(meta.shape[0] -1)
    meta["CellID"] = meta["Barcode"]
    meta.drop("Barcode", inplace=True, axis=1)
    meta.index = meta["CellID"]
    adata_template.obs = adata_template.obs.join(meta, how="left")
    adata_template = adata_template[adata_template.obs["orig.ident"] == "ST_OSCC"]
    return adata_template

def get_genes(adata_template, ic50_dgidb, i, all_genes):
    # Get genes_up, genes_down, and nz_valid_genes
    nonzero_input = adata_template.layers["spliced"]
    nz_input_genesums = nonzero_input.sum(axis=0)
    valid_ind = np.logical_and(np.isfinite(nz_input_genesums), nz_input_genesums != 0)
    valid_ind = np.array(valid_ind).flatten()
    bad_genes = adata_template.var[~valid_ind]
    valid_genes = set(all_genes) - (set(all_genes) - set(adata_template.var.index.to_series().tolist()))
    nz_valid_genes = valid_genes - set(bad_genes.index.tolist()).intersection(valid_genes)
    
    genes_up = ic50_dgidb.loc[i]['genes_up']
    genes_down = ic50_dgidb.loc[i]['genes_down']

    genes_up = str(genes_up).replace('nan', '').split(" ")
    genes_up = set(genes_up).intersection(nz_valid_genes)

    genes_down = str(genes_down).replace('nan', '').split(" ")
    genes_down = set(genes_down).intersection(nz_valid_genes)

    if '' in genes_down:
        genes_down.remove('')

    if '' in genes_up:
        genes_up.remove('')
        
    nz_valid_genes = adata_template.var.index.to_series().tolist()
    
    return genes_up, genes_down, nz_valid_genes

def run_perturbation_analysis(adata_pt, genes_up, genes_down, nz_valid_genes, i, ic50_dgidb, ic50_dgidb_scored):
    genes_total = list(genes_down) + list(genes_up)

    dyn.pp.recipe_monocle(adata_pt, genes_to_append=genes_total)
    dyn.tl.dynamics(adata_pt, cores=9)

    dyn.tl.reduceDimension(adata_pt)

    dyn.tl.cell_velocities(adata_pt, basis='pca')
    dyn.tl.cell_velocities(adata_pt, basis='umap')

    dyn.vf.VectorField(adata_pt, basis='pca', M=50)
    dyn.vf.VectorField(adata_pt, basis='umap', M=50)

    ic50_dgidb_scored = perturb_condition(adata_pt, genes_up, genes_down, genes_total, i, ic50_dgidb, ic50_dgidb_scored)
    
    return ic50_dgidb_scored

def perturb_condition(adata_pt, genes_up, genes_down, genes_total, i, ic50_dgidb, ic50_dgidb_scored):
    if (len(genes_up) >= 1 and len(genes_down) >= 1):
        print("up and down")
        expr_vals = [-200] * len(genes_down) + [200] * len(genes_up)
    elif (len(genes_up) >= 1 and len(genes_down) == 0):
        print("only up")
        expr_vals = [200] * len(genes_up)
    elif (len(genes_up) == 0 and len(genes_down) >= 1):
        print("only down")
        expr_vals = [-200] * len(genes_down)
    else:
        print("no perturb")
        return ic50_dgidb_scored

    ic50_dgidb_scored = run_perturb_analysis(adata_pt, genes_total, expr_vals, i, ic50_dgidb, ic50_dgidb_scored)
    return ic50_dgidb_scored


def run_perturb_analysis(adata_pt, genes_total, expr_vals, i, ic50_dgidb, ic50_dgidb_scored):
    try:
        print(i)
        print(expr_vals)
        print(genes_total)
        dyn.pd.perturbation(adata_pt, genes_total, expr_vals, emb_basis="umap")
        dyn.vf.VectorField(adata_pt, basis='umap_perturbation', M=50)
        print("Generating state graph")

        dyn.pd.state_graph(adata_pt,group='cluster_annotations', basis='umap_perturbation',transition_mat_key='perturbation_transition_matrix')
        
        dyn.pl.streamline_plot(adata_pt, basis = "umap_perturbation",
                          color=["cluster_annotations"],
                          color_key = ["#4DBBD5FF","#E64B35FF","#F9E076"],
                          show_legend = False,
                          calpha = 1,
                          pointsize = 0.1,
                          save_show_or_return = "save",
                           save_kwargs = {"path": f"plots/state_graph/{i}_vector", "prefix": 'scatter', "dpi": 300, "ext": 'png', 
                           "transparent": False, "close": True, "verbose": True},
                          figsize=(5,5),
                          dpi = 300)
        
        Pl = adata_pt.uns["cluster_annotations_graph"]["group_graph"]
        flat_array = Pl.flatten()
        ic50_dgidb_scored.at[i, "pertrub_mtx"] = list(flat_array)

        print("making state graph plot")
        dyn.pl.state_graph(adata_pt,
                           color=['cluster_annotations'],
                           color_key=["#4DBBD5FF", "#E64B35FF", "#F9E076"],
                           group='cluster_annotations',
                           basis='umap_perturbation',
                           save_show_or_return='save',
                           save_kwargs={"path": f"plots/state_graph/{i}_sg", "prefix": 'scatter', "dpi": 300, "ext": 'png',
                                        "transparent": False, "close": True, "verbose": True},
                           keep_only_one_direction=False,
                           figsize=(5, 5))
        return ic50_dgidb_scored
    except:
        return ic50_dgidb_scored

In [14]:
ic50_dgidb_scored = process_data(ic50_dgidb_to_process, all_genes)

|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...
|-----> <insert> pp to uns in AnnData Object.
|-----------> <insert> has_splicing to uns['pp'] in AnnData Object.
|-----------> <insert> has_labling to uns['pp'] in AnnData Object.
|-----------> <insert> splicing_labeling to uns['pp'] in AnnData Object.
|-----------> <insert> has_protein to uns['pp'] in AnnData Object.
|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----------> <insert> tkey to uns['

Transforming subset Jacobian: 100%|███████| 13949/13949 [00:00<00:00, 117381.27it/s]


|-----> <insert> j_delta_x_perturbation to obsm in AnnData Object.
|-----> project the pca perturbation vector to low dimensional space....
|-----> 0 genes are removed because of nan velocity values.
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%
|-----> [projecting velocity vector to low dimensional embedding] finished [1.8918s]
|-----> <insert> X_umap_perturbation to obsm in AnnData Object.
|-----> you can use dyn.pl.streamline_plot(adata, basis='umap_perturbation') to visualize the perturbation vector
|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: UMAP_PERTURBATION. 
        Vector field will be learned in the UMAP_PERTURBATION space.
|-----> Generating high dimensional grids and convert into a row matrix.
|-----> Learning vector field with method: sparsevfc.
|-----> [SparseVFC] begins...
|-----> Sampling control points based on data velocity magnitude...
|-----> [SparseVFC] in progress: 100.0000%
|----

integration with ivp solver: 100%|███████████████| 100/100 [00:00<00:00, 420.64it/s]
uniformly sampling points along a trajectory: 100%|█| 100/100 [00:00<00:00, 351.73it


|-----> [iterate groups] in progress: 66.6667%

...

|-----> [iterate groups] in progress: 66.6667%

integration with ivp solver: 100%|███████████████| 100/100 [00:00<00:00, 455.88it/s]
uniformly sampling points along a trajectory: 100%|█| 100/100 [00:00<00:00, 381.99it


|-----> [iterate groups] in progress: 100.0000%

integration with ivp solver: 100%|███████████████| 100/100 [00:00<00:00, 383.56it/s]
uniformly sampling points along a trajectory: 100%|█| 100/100 [00:00<00:00, 361.26it


|-----> [iterate groups] in progress: 100.0000%
|-----> [iterate groups] finished [2.2511s]
|-----> [State graph estimation] in progress: 100.0000%
|-----> [State graph estimation] finished [0.0014s]
|-----------> plotting with basis key=X_umap_perturbation
|-----------> skip filtering cluster_annotations by stack threshold when stacking color because it is not a numeric type
Saving figure to plots/state_graph/scatter_Pazopanib_vector.png...
Done
making state graph plot
|-----------> plotting with basis key=X_umap_perturbation
|-----------> skip filtering cluster_annotations by stack threshold when stacking color because it is not a numeric type
Saving figure to plots/state_graph/scatter_Pazopanib_sg.png...
Done


In [ ]:
ic50_dgidb_scores = ic50_dgidb_scored[ic50_dgidb_scored['pertrub_mtx'].apply(lambda x: x != [0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [ ]:
ic50_dgidb_scores['edge_outgoing'] = [x+y for x,y in zip([el[3] for el in ic50_dgidb_scores['pertrub_mtx']],
                                     [el[5] for el in ic50_dgidb_scores['pertrub_mtx']])]

ic50_dgidb_scores['edge_incoming'] = [x+y for x,y in zip([el[1] for el in ic50_dgidb_scores['pertrub_mtx']],
                                     [el[7] for el in ic50_dgidb_scores['pertrub_mtx']])]

ic50_dgidb_scores['core_incoming'] = [x+y for x,y in zip([el[2] for el in ic50_dgidb_scores['pertrub_mtx']],
                                     [el[5] for el in ic50_dgidb_scores['pertrub_mtx']])]

ic50_dgidb_scores['core_outgoing'] = [x+y for x,y in zip([el[6] for el in ic50_dgidb_scores['pertrub_mtx']],
                                     [el[7] for el in ic50_dgidb_scores['pertrub_mtx']])]

ic50_dgidb_scores['core_net'] = ic50_dgidb_scores['core_incoming']- ic50_dgidb_scores['core_outgoing'] 
ic50_dgidb_scores['edge_net'] = ic50_dgidb_scores['edge_incoming']- ic50_dgidb_scores['edge_outgoing'] 

In [ ]:
ic50_dgidb_scores.to_csv("all_200_rescaled_vf.csv")

In [8]:
import seaborn as sns

In [12]:
sns.boxplot(x='AUC_category', y='core_outgoing', data=ic50_dgidb_scores)
sns.stripplot(x='AUC_category', y='core_outgoing', data=ic50_dgidb_scores, jitter=True, color='black')
sns.despine()

In [13]:
sns.lmplot(x="AUC_mean", y="core_outgoing", data=ic50_dgidb_scores)